In [1]:
import re
import requests
import math
from lxml import html
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 計算 org+title+per, org+per, org, per 的詞頻
- ner區：某些詞組不到、group ner頻率通常是1、會受組合的dataframe順序影響最後排序結果
- 文章區：

In [2]:
def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
        print(idx, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
        except Exception as e:
            print(e)
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text, getLink

def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [13]:
## Data Format
def ner_combine(ner_pair):
    for idx, (word, ner, cat) in enumerate(ner_pair):
        if (idx + 1) == len(ner_pair) or (idx + 2) == len(ner_pair):
            break
        else:  # 直接將組合詞加入原先分詞好的list (ner_pair)
            n1_ner = ner_pair[idx+1]
            if cat == n1_ner[2]:
                n2_ner = ner_pair[idx+2]
                if ner == 'ORG' and n1_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0],'ORG+PER', cat)
                    del ner_pair[idx:idx+2]
                    ner_pair.insert(idx,new_wd)
                elif ner == 'ORG' and n1_ner[1] == 'TITLE' and n2_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0]+n2_ner[0],'ORG+TITLE+PER', cat)
                    del ner_pair[idx:idx+3]
                    ner_pair.insert(idx,new_wd)

def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    ner_wd = []
    trim = stopwords()
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        ner_wd.append(" ".join(str(x) for x in temp))
    ner_combine(ner_pair)
    return ner_pair, ner_wd

In [54]:
## ner區
def auto_extract(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'PER'], 
                          df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG']), ignore_index=True)
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.words).most_common(15),columns=['words', 'cnt'])
    return cnt

## 文章區
def article(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'PER'], 
                          df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'],
                         df[df['ner'] == 'TITLE'])).sort_index()
    keywords = keywords.reset_index()
    keywords = keywords.drop(columns =['index'])
    cnt = auto_extract(ner_pair)
    cntList = cnt.words.unique().tolist()
    
    idx = [i for i in range(len(keywords)) if keywords.words.iloc[i] in cntList]
    keywords = keywords.drop(idx).reset_index()
    keywords = keywords.drop(columns=['index', 'cat'])
    return keywords    

In [55]:
## commands
url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'] # select one url
final_text, link = getFinalText(url)
ner_pair, ner_wd = ner_extract(final_text)
cnt = auto_extract(ner_pair)
keywords = article(ner_pair)

0 http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html


## 計算 PER 的詞頻
- NER: 難以分辨每個人名的重要性
- Content: need to rethink

In [17]:
def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
        print(idx, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
        except Exception as e:
            print(e)
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text, getLink

def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [3]:
## Data Format
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    ner_wd = []
    trim = stopwords()
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        ner_wd.append(" ".join(str(x) for x in temp))
    return ner_pair, ner_wd

In [29]:
## ner區
def auto_extract(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = df[df['ner'] == 'PER']
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.words).most_common(),columns=['words', 'cnt'])
    return cnt

## 文章區
def article(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = pd.concat((df[df['ner'] == 'PER'], df[df['ner'] == 'ORG'], df[df['ner'] == 'TITLE'])).sort_index()
    keywords = keywords.reset_index()
    keywords = keywords.drop(columns =['index', 'cat'])
#     cnt = auto_extract(ner_pair)
#     cntList = cnt.words.unique().tolist()
    
#     idx = [i for i in range(len(keywords)) if keywords.words.iloc[i] in cntList]
#     keywords = keywords.drop(idx).reset_index()
#     keywords = keywords.drop(columns=['index', 'cat'])
    return keywords    

In [31]:
## commands
url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'] # select one url
final_text, link = getFinalText(url)
ner_pair, ner_wd = ner_extract(final_text)
cnt = auto_extract(ner_pair)
keywords = article(ner_pair)

0 http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html


## 計算ORG的詞頻
- NER: 覺得只計算組織的詞頻沒太多用處
- Content: need to rethink

In [33]:
def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
        print(idx, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
        except Exception as e:
            print(e)
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text, getLink

def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [35]:
## Data Format
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    ner_wd = []
    trim = stopwords()
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        ner_wd.append(" ".join(str(x) for x in temp))
    return ner_pair, ner_wd

In [36]:
## ner區
def auto_extract(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = df[df['ner'] == 'ORG']
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.words).most_common(),columns=['words', 'cnt'])
    return cnt

## 文章區 -- need to rethink
def article(ner_pair):
    df = pd.DataFrame(ner_pair,columns=['words','ner','cat'])  # 選定文章的分詞結果
    keywords = pd.concat((df[df['ner'] == 'PER'], df[df['ner'] == 'ORG'], df[df['ner'] == 'TITLE'])).sort_index()
    keywords = keywords.reset_index()
    keywords = keywords.drop(columns =['index', 'cat'])
    return keywords    

In [37]:
## commands
url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'] # select one url
final_text, link = getFinalText(url)
ner_pair, ner_wd = ner_extract(final_text)
cnt = auto_extract(ner_pair)
keywords = article(ner_pair)

0 http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html


## 計算特定NER的 TFIDF
1. PER
2. ORG
3. ORG & PER

In [2]:
def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點 (無數字)
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
        print(idx +1, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('無結果')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
            a_tag = soup.find_all('a')
            for a in a_tag:
                a.decompose() 
            html_text.append(soup.get_text()) 
        except Exception as e:
            print(e)
            html_text.append('無結果') 
                 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 只保留漢字、數字、大小寫英文
    temp1 = [re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039])","", texts) for texts in temp]

    # 去除停用詞 (先去除還是等到分詞時再去除)
    final_text = [multi_repl(texts) for texts in temp1]
    
    # 紀錄資訊量少的文章 (文字少於400者)
    lessInfo = [i+1 for i in range(len(final_text)) if len(final_text[i]) < 400]
    
    return final_text, link

In [ ]:
## Data Format
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    ner_wd = []
    trim = stopwords()
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        ner_wd.append(" ".join(str(x) for x in temp))
    return ner_pair, ner_wd

In [70]:
def getIdx(url):
    df = pd.DataFrame(getLink, columns=['idx', 'link'])
    for i in range(len(df)):
        if df.iloc[i,1] == url[0]:
            return i

In [111]:
## ner區
def auto_extract(doc_terms_freq):
    wd = list(df_specific[df_specific['cat']==getIdx(url)+1]['words'])
    df_tfidf = pd.DataFrame(columns=['word','tfidf'])
    for i, count in enumerate(doc_terms_freq):
        if i == getIdx(url):
            for word in count:
                if word in wd:
                    df_tfidf = df_tfidf.append({'word':word,'tfidf': tf(word,count)*idf(word,doc_terms_freq)},
                                               ignore_index=True)
    df_tfidf = df_tfidf.sort_values(by=['tfidf'], ascending=False).reset_index()
    df_tfidf = df_tfidf.drop(columns=['index'])  ## 程式產出結果
    return df_tfidf[:15]

In [124]:
## calculate idf
def needs():
    google_search_rs = pd.read_excel('ner_sample.xlsx').url.to_list()
    google_search_text, getLink = getFinalText(google_search_rs)
    ner_pair_S, ner_wd_S = ner_extract(google_search_text)

    df_all_pair = pd.DataFrame(ner_pair_S,columns=['words','ner','cat'])
    
#     df_specific = pd.concat((df_all_pair[df_all_pair['ner'] == 'ORG'], df_all_pair[df_all_pair['ner'] == 'PER'])).sort_index()
#     df_specific = df_specific.reset_index()

    df_specific = df_all_pair[df_all_pair['ner'] == 'PER'].reset_index()  # select PER or ORG
    
    df_specific = df_specific.drop(columns=['index'])

    doc_terms_freq = []  # 每篇文章每個詞的頻率
    for i in range(len(google_search_text)):
        tempDF = df_all_pair[df_all_pair['cat']==i+1]
        cnt = Counter(tempDF.words)
        doc_terms_freq.append(cnt)
    return df_specific, doc_terms_freq

In [107]:
#计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())

# 计算idf
def idf(word, count_list):
    return math.log(len(count_list) / (n_containing(word, count_list)))  

In [125]:
## commands
df_specific, doc_terms_freq = needs() # needs for calculating idf

url = ['http://www.shio.gov.cn/sh/xwb/n809/n814/n1061/u1ai18626.html'] # select one url
final_text, link = getFinalText(url)
ner_pair, ner_wd = ner_extract(final_text)

auto_extract(doc_terms_freq)

2021-06-15 16:39:35,123 - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


,word,tfidf
0,北京科学技术情报学会,0.044125
1,北京市科学技术协会,0.026475
2,中国科学技术信息研究所,0.017650
3,戴国强,0.017650
4,吴晨生,0.017650
5,中国科协,0.017650
6,辽宁省科学技术情报研究所,0.008825
7,科技部中国科技发展战略研究院调查统计中心,0.008825
8,军事科学院国防科技创新研究院,0.008825
9,栗琳,0.008825


## 字典

In [ ]:
def add_dict(word, ner):
    path = '/Users/csti-user/Downloads/NER/custom_dict.txt'
    with open(path,'a') as f:
        f.write('%s/%s\n'%(word,ner)) 

In [ ]:
add_dict('資策會', 'ORG')